In [1]:
%pip install ollama pypdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install "langchain-chroma>=0.1.2"

  Using cached langchain_chroma-0.2.3-py3-none-any.whl.metadata (1.1 kB)
  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6-cp313-cp313-win_amd64.whl
  Using cached onnxruntime-1.21.1-cp313-cp313-win_amd64.whl.metadata (4.9 kB)
  Using cached opentelemetry_api-1.32.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.32.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.53b1-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_sdk-1.32.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached grpcio-1.71.0-cp313-cp313-win_a


[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install -qU langchain-ollama

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from langchain_community.utils.math import cosine_similarity
from langchain_ollama import OllamaEmbeddings
from langchain_ollama.llms import OllamaLLM
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
import ollama
from pypdf import PdfReader
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")



In [7]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma

llmgpt4o = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [8]:
def getVectorStorage(collectionName, embeddingModel):
    return Chroma(
        collection_name= collectionName,
        embedding_function=embeddingModel,
        persist_directory='./chroma_exp'
    )

In [9]:
def getTextData(pdfPath):
    reader = PdfReader(pdfPath)
    allText = ''
    for i in range(len(reader.pages)):
        read = reader.pages[i]
        allText += read.extract_text() + '\n'
    return allText

In [10]:
def getFromVector(query, vector_store):
    docs = vector_store.similarity_search(query, k=5) 
    return docs

In [11]:
def generateAnswer(query, vector_store, chat_llm):
    relevant_docs = getFromVector(query, vector_store)
    context = "\n".join([doc.page_content for doc in relevant_docs])

    prompt = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
    response = chat_llm.invoke(prompt)
    return response

In [12]:
def PdfToChroma(text, vector_store):
    print("Storing in ",vector_store._collection_name)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    texts = text_splitter.split_text(text)
    
    docs = [Document(page_content=chunk) for chunk in texts]
    vector_store.add_documents(docs)
    print("PDF Stored Successfully")
    return "PDF Stored Successfully"

In [ ]:
AllTestingDocuments = [
    # "E-Assessment Tools for Programming Languages.pdf",
    # "Analysis_Of_Medical_Data_Using_Data_Mini.pdf",
    # "GuptaKB05.pdf",
    # "191-40-25--.pdf",
    # "978-981-96-6525-9.pdf",
    # "Article-FilePdf-202412311195182.pdf",
    # "ComparisonofNeuralNetworks.pdf",
    # "Extractionofcollagenfromby.pdf",
    # "Fake_News_Detection.pdf",
    # "IJIRCT2411033.pdf",
    "RTP_ArtificialIntelligenceinE-Government.pdf",
    "The Role of Big Data in Smart Healthcare.pdf",
    "TheCriticalRoleofDataScience.pdf",
    "TheRoleofMachineLearninginBlockchainScalability.pdf"
]

In [53]:
StandardQuestions = [
    "What is the main research problem or question addressed in the paper?",
    "What methodology or approach has been used to solve the problem or answer the question?",
    "What are the key findings or results of the study?",
    "What are the limitations of the research mentioned in the paper?",
    "How does this research build upon or differ from previous studies in the field?",
    "What data or evidence has been used, and how was it collected or analyzed?",
    "What are the implications or practical applications of the findings?",
    "What assumptions are made in the study, and are they justified?",
    "Are there any unanswered questions or suggested areas for future research?",
    "How does the research contribute to its field, and why is it important?"
]

In [54]:
def getSummaryFromLLM(pdfPath, chat_llm):
    textData = getTextData(pdfPath)
    response = chat_llm.invoke(f"Please generate a detailed summary of the provided text, ensuring all key facts and information are included. The summary should be no less than 2 pages and no more than 5 pages in length.\n\nText:\n\n{textData}")
    return response

In [55]:
def getVectorEmbed(query):
    return embeddings.embed_query(query)

def getCosineSimilarity(summary_vector, standard_vector):
    return cosine_similarity([summary_vector], [standard_vector])

In [56]:
import csv

In [57]:
def computeResult(model, vector_store_summary, vector_store_raw, chat_llm, pdfPath):
    raw_document_text = getTextData(pdfPath)
    summary = getSummaryFromLLM(pdfPath, chat_llm)

    PdfToChroma(raw_document_text, vector_store_raw)
    PdfToChroma(summary, vector_store_summary)
    res_data = [
        ["model", "vector_store", "type", "question", "answer"]
    ]

    stat_data = [
        ["question", "similarity", "knowledgescore" "model", "pdf"]
    ]

    for question in StandardQuestions:
        response_summary = generateAnswer(question, vector_store_summary, chat_llm)
        response_raw = generateAnswer(question, vector_store_raw, chat_llm)
        print("Question: ", question)
        result = llmgpt4o.invoke(f"Question: {question}\n\nDocument Answer: {response_raw}\n\nSummary Answer: {response_summary}\n\nQuery: Compute how similar the answers are and provide a score where 100 is the highest and 0 is the lowest. Scoring must be based on the 1. Relevance, 2. Coherence, 3. Conciseness, 4. Factual Consistency and quality, facts and similar meaning of the answers. No Additional Information is required. Output must be like  - Score: [[82.4]]")
        knowledgescore = result.content.partition('[[')[2].partition(']]')[0]
        print("Knowledge Score: ", knowledgescore)
        similarity = getCosineSimilarity(getVectorEmbed(response_summary), getVectorEmbed(response_raw))
        print("Cosine Similarity: ", similarity)
        res_data.append([model, vector_store_summary._collection_name, "summary", question, response_summary])
        res_data.append([model, vector_store_raw._collection_name, "raw", question, response_raw])
        stat_data.append([question, similarity, knowledgescore, model, pdfPath])


    with open(f'{model}-{''.join([x for x in pdfPath if x.isalnum()])}-results.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(res_data)

    with open(f'{model}-{''.join([x for x in pdfPath if x.isalnum()])}-stats.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(stat_data)

In [65]:
chat_llm = OllamaLLM(model="llama3.1")
model = "llama3-1"


In [79]:

for pdf in AllTestingDocuments:
    print("pdf: ", pdf)
    vector_store_raw = getVectorStorage(f"Raw_{''.join([x for x in pdf if x.isalnum()])}_{model}", embeddings)
    vector_store_summary = getVectorStorage(f"Summary_{''.join([x for x in pdf if x.isalnum()])}_{model}", embeddings)
    vector_store_raw.reset_collection()
    vector_store_summary.reset_collection()
    computeResult(model, vector_store_summary, vector_store_raw, chat_llm, pdf)

pdf:  Fake_News_Detection.pdf
Storing in  Raw_FakeNewsDetectionpdf_llama3-1
PDF Stored Successfully
Storing in  Summary_FakeNewsDetectionpdf_llama3-1
PDF Stored Successfully


Number of requested results 5 is greater than number of elements in index 2, updating n_results = 2


Question:  What is the main research problem or question addressed in the paper?
Knowledge Score:  85.7
Cosine Similarity:  [[0.87053027]]


Number of requested results 5 is greater than number of elements in index 2, updating n_results = 2


Question:  What methodology or approach has been used to solve the problem or answer the question?
Knowledge Score:  75.0
Cosine Similarity:  [[0.73529735]]


...

Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4


Question:  Are there any unanswered questions or suggested areas for future research?
Knowledge Score:  85.0
Cosine Similarity:  [[0.8291577]]


Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4


Question:  How does the research contribute to its field, and why is it important?
Knowledge Score:  88.1
Cosine Similarity:  [[0.92205394]]
pdf:  RTP_ArtificialIntelligenceinE-Government-IdentifyingandAddressingKeyChallenges.pdf


ValueError: Expected collection name that (1) contains 3-63 characters, (2) starts and ends with an alphanumeric character, (3) otherwise contains only alphanumeric characters, underscores or hyphens (-), (4) contains no two consecutive periods (..) and (5) is not a valid IPv4 address, got Raw_RTPArtificialIntelligenceinEGovernmentIdentifyingandAddressingKeyChallengespdf_llama3-1